# LecoSpec Data Munging

In [105]:
# notebooks use their location as their working directory, so
# if we are in a subfolder, move to the main folder.  
# This however can safely be run multiple times
#setwd(M:/lecospec/lecospec)
if(!dir.exists("Functions/")){
    setwd("../")
    if(!dir.exists("Functions")){
        setwd("M:/lecospec/lecospec/")
    }
}
source("Functions/lecospectR.R", echo = FALSE)

## Notation

Throughout the notebook, variables starting with `img_` are UAV image-based information (data, filepaths, etc).  Similarly, variables beginning with `grd_` related to data collected on the ground.  

Also, some other naming conventions for variables with data transformations:
* `robust` in a variable name refers to data treated by center according to the median and scaling by teh inter-quartile range (a la sklearns RobustScaler)
* `minmax` (and its ilk) are min-max scaled data, i.e. scaled to the interval [0,1] by subtracting the minimum and dividing by the range.
* `standard(ized)` refers to data treated with with the z-score transform by centring using the mean and scaling y the standard deviation (like sklearns StandardScaler)
* `corrected` means that a linear transformation has been applied to account for differences in sensor calibration.
* `raw` refers to having no transformations applied
* `clipped` means that outliers have been clipped to the upper and lower fence values based on the Inter-Quartile Range method. 
* `imputed` means that outliers have been removed and imputed
* `dropped` means that dataframe rows containing outliers have been removed

Example: `img_robust_indices` refers to vegetation indices from the UAV images treated with the robust scaler. 

## Define data locations


In [106]:
# spectral library
#grd_base_path <- "./Output/C_001_SC3_Cleaned_SpectralLib.csv"
#grd_base_path <- "./Output/C_001_SC3_Cleaned_SpectralLib_CommonSp.csv"
grd_base_path <- "./Output/C_001_SC3_Cleaned_SpectralLib_CenAkCommonSp.csv"

grd_speclib <- read.csv(grd_base_path, header = TRUE)
#grd_index_path <- ./Data/D_002_SpecLib_Derivs.csv
#grd_indices <- read.csv(grd_index_path)
# this data has some lines that have no labels, so we remove them 
grd_speclib <- grd_speclib[!is.na(grd_speclib$Functional_group1),]
head(grd_speclib)
grd_speclib %>% group_by(Area,Functional_group1) %>% tally %>% pivot_wider(values_from = n, names_from = Functional_group1)



,X.1,X,ScanID,Area,Code_name,Species_name,Functional_group1,Functional_group2,Species_name_Freq,Functional_group1_Freq,⋯,X2491,X2492,X2493,X2494,X2495,X2496,X2497,X2498,X2499,X2500
,<int>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<int>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,58,58,arcrub_Murph_236,Murphy,arcrub,Arctostaphyllos,ShrubDecid,ShrubDecidOther,17,360,⋯,8.4147,8.3791,8.3507,8.3295,8.3210,8.3194,8.3127,8.3032,8.2997,8.3031
2,59,59,arcrub_Murph_237,Murphy,arcrub,Arctostaphyllos,ShrubDecid,ShrubDecidOther,17,360,⋯,11.1266,11.1174,11.1209,11.1233,11.1035,11.0761,11.0640,11.0635,11.0625,11.0598
3,60,60,arcrub_Murph_238,Murphy,arcrub,Arctostaphyllos,ShrubDecid,ShrubDecidOther,17,360,⋯,7.4801,7.4304,7.3736,7.3171,7.2712,7.2321,7.1991,7.1727,7.1627,7.1732
4,61,61,arcrub_Murph_239,Murphy,arcrub,Arctostaphyllos,ShrubDecid,ShrubDecidOther,17,360,⋯,6.7298,6.7142,6.6879,6.6496,6.5895,6.5199,6.4662,6.4247,6.4019,6.4002
5,62,62,arcrub_Murph_240,Murphy,arcrub,Arctostaphyllos,ShrubDecid,ShrubDecidOther,17,360,⋯,7.3555,7.3651,7.3673,7.3604,7.3327,7.2958,7.2703,7.2531,7.2513,7.2666
6,63,63,arcrub_Murph_241,Murphy,arcrub,Arctostaphyllos,ShrubDecid,ShrubDecidOther,17,360,⋯,4.9941,4.9993,4.9919,4.9790,4.9613,4.9405,4.9216,4.9041,4.8951,4.8952


Area,Lichen,ShrubEvergreen,ShrubDecid,TreeConifer,Graminoid,Moss,TreeBroadleaf,Abiotic,Forb
<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
12mile,16,9,NA,NA,NA,NA,NA,NA,NA
Big Trail,33,7,12,9,NA,NA,NA,NA,NA
BirchLake,NA,NA,10,NA,NA,NA,NA,NA,NA
Brooks_Range,35,13,8,NA,2,8,4,NA,NA
Eight Mile,NA,NA,NA,NA,NA,5,NA,NA,NA
Fairbanks area,8,4,8,8,NA,12,NA,13,4
Fairbanks Area Spinach Circle,NA,2,10,NA,NA,6,NA,NA,NA
Fairbanks Area Vault Road,NA,NA,NA,NA,NA,NA,9,NA,NA
Murphy,100,2,60,NA,NA,23,NA,20,33


In [107]:
img_base_path <- "Data/Ground_Validation/PFT_Image_spectra/PFT_Image_SpectralLib_Clean.csv"
img_speclib <- read.csv(img_base_path, header = TRUE)
# currently, not using the old pre-proccessing scheme and just doing it here.
#img_index_path <- Data/D_002_Image_SpecLib_Derivs.csv
#img_speclib <- read.csv(img_base_path)
head(img_speclib)

,X,UID,ScanNum,sample_name,PFT,FncGrp1,Site,X398,X399,X400,⋯,X991,X992,X993,X994,X995,X996,X997,X998,X999,Area
,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
1,1,BisonGulchPFTsBetula1,1,spec_1,Betula,TreeBroadleaf,BisonGulch,0.05814769,0.05926529,0.06028869,⋯,0.6811660,0.6890470,0.7040298,0.7249807,0.7507566,0.7801884,0.8121027,0.8453261,0.8786852,BisonGulch
2,2,BisonGulchPFTsBetula1,1,spec_2,Betula,TreeBroadleaf,BisonGulch,0.04456014,0.04778814,0.05079318,⋯,0.6683159,0.6786394,0.7000307,0.7308801,0.7695067,0.8140391,0.8625739,0.9132079,0.9640378,BisonGulch
3,3,BisonGulchPFTsBetula1,1,spec_3,Betula,TreeBroadleaf,BisonGulch,0.03929324,0.04265593,0.04557066,⋯,0.5091915,0.5178217,0.5395294,0.5726982,0.6156166,0.6663192,0.7227978,0.7830447,0.8450520,BisonGulch
4,4,BisonGulchPFTsBetula1,1,spec_4,Betula,TreeBroadleaf,BisonGulch,0.13230228,0.11122692,0.09129034,⋯,0.5113880,0.5348292,0.5745538,0.6227243,0.6723311,0.7185860,0.7570701,0.7833644,0.7930498,BisonGulch
5,5,BisonGulchPFTsBetula1,1,spec_5,Betula,TreeBroadleaf,BisonGulch,0.05211388,0.05565497,0.05878525,⋯,0.6680365,0.6509006,0.6344450,0.6181806,0.6017555,0.5851848,0.5685449,0.5519121,0.5353626,BisonGulch
6,6,BisonGulchPFTsBetula1,1,spec_6,Betula,TreeBroadleaf,BisonGulch,0.06955397,0.06788242,0.06631141,⋯,0.7371508,0.7445194,0.7567953,0.7732173,0.7930235,0.8154512,0.8397375,0.8651196,0.8908347,BisonGulch


Okay, there are some metadata columns that should not be there for the next step - lets remove them with `subset`

In [108]:
img_bands <- subset(
    img_speclib, 
    select=-c(
        X,
    	UID,
        ScanNum,
    	sample_name,
    	PFT,
    	FncGrp1
    ))


grd_bands <- subset(
    grd_speclib, 
    select=-c(
        X,
        ScanID,
        Area,
        Code_name,
        Species_name,
        Functional_group1,
        Functional_group2,
        Species_name_Freq,
        Functional_group1_Freq,
        Functional_group2_Freq,
        Genus,
        Version,
        File.Name,
        Instrument,
        Detectors,
        Measurement,
        Date,
        Time,
        Battery.Voltage,
        Averages,
        Integration1,
        Integration2,
        Integration3,
        Dark.Mode,
        Foreoptic,
        Radiometric.Calibration,
        Units,
        Latitude,
        Longitude,
        Altitude,
        GPS.Time,
        Satellites,
        Calibrated.Reference.Correction.File,
        Channels,
        ScanNum
        
        
    )
)

Calculate the vegetation indices from the spectral libraries - its easy with lecospectR!

Note that the image-based scpectra are normalized from zero to one, and the ground specctra are on the range zero to one hundred.  

In [109]:
img_indices <- get_vegetation_indices(img_bands, NULL)# should have a default of NULL, you know?
grd_indices <- get_vegetation_indices(grd_bands, NULL)

In [110]:
write.csv(img_indices, file="Data/gs/x_train/img_indices_only.csv")

write.csv(grd_indices, file="Data/gs/x_train/grd_indices_only.csv")

In [111]:
head(img_indices)
head(img_indices)

,Boochs,Boochs2,CARI,Carter,Carter2,Carter3,Carter4,Carter5,Carter6,CI,⋯,TCARI,TCARIOSAVI,TCARI2,TCARI2OSAVI2,TGI,TVI,Vogelmann,Vogelmann2,Vogelmann3,Vogelmann4
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,0.01118062,0.01252303,0.5520621,2.540064,0.2065954,0.1994095,0.4599336,1.640920,0.1548045,1.212547,⋯,0.2615574,0.3456065,0.4178572,0.9235753,8.286956,35.80558,1.439152,-0.08239895,1.064248,-0.09009970
2,0.01111790,0.01355575,0.5766599,2.625150,0.2065916,0.2070232,0.4513357,1.656177,0.1700914,1.175201,⋯,0.2831688,0.3720026,0.4496971,0.9654107,9.286506,38.17785,1.446272,-0.08680580,1.135065,-0.09518428
3,0.01084760,0.01378545,0.5614974,3.251237,0.1987500,0.2026890,0.4400645,1.585012,0.1803148,1.173642,⋯,0.2865432,0.3797346,0.4659473,0.9777083,10.063824,38.92187,1.454211,-0.07940521,1.153111,-0.08775410
4,0.01181043,0.01221932,0.5747244,2.602032,0.1809264,0.1767072,0.4379739,1.650827,0.1572204,1.084016,⋯,0.2773505,0.3524937,0.4235281,0.8672684,8.906298,40.04949,1.449666,-0.10812234,1.245379,-0.11775728
5,0.01119492,0.01259771,0.4934165,2.215177,0.1763305,0.1782588,0.4198506,1.671405,0.1517899,1.214389,⋯,0.2530670,0.3235553,0.3943958,0.7973731,8.621081,37.23711,1.486982,-0.09404361,1.060612,-0.10349363
6,0.01124955,0.01239871,0.5616888,2.578908,0.2044972,0.1928170,0.4591589,1.658333,0.1515471,1.234711,⋯,0.2609731,0.3423476,0.4145336,0.9106479,8.084553,36.27321,1.438798,-0.08098956,1.066427,-0.08849283


,Boochs,Boochs2,CARI,Carter,Carter2,Carter3,Carter4,Carter5,Carter6,CI,⋯,TCARI,TCARIOSAVI,TCARI2,TCARI2OSAVI2,TGI,TVI,Vogelmann,Vogelmann2,Vogelmann3,Vogelmann4
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,0.01118062,0.01252303,0.5520621,2.540064,0.2065954,0.1994095,0.4599336,1.640920,0.1548045,1.212547,⋯,0.2615574,0.3456065,0.4178572,0.9235753,8.286956,35.80558,1.439152,-0.08239895,1.064248,-0.09009970
2,0.01111790,0.01355575,0.5766599,2.625150,0.2065916,0.2070232,0.4513357,1.656177,0.1700914,1.175201,⋯,0.2831688,0.3720026,0.4496971,0.9654107,9.286506,38.17785,1.446272,-0.08680580,1.135065,-0.09518428
3,0.01084760,0.01378545,0.5614974,3.251237,0.1987500,0.2026890,0.4400645,1.585012,0.1803148,1.173642,⋯,0.2865432,0.3797346,0.4659473,0.9777083,10.063824,38.92187,1.454211,-0.07940521,1.153111,-0.08775410
4,0.01181043,0.01221932,0.5747244,2.602032,0.1809264,0.1767072,0.4379739,1.650827,0.1572204,1.084016,⋯,0.2773505,0.3524937,0.4235281,0.8672684,8.906298,40.04949,1.449666,-0.10812234,1.245379,-0.11775728
5,0.01119492,0.01259771,0.4934165,2.215177,0.1763305,0.1782588,0.4198506,1.671405,0.1517899,1.214389,⋯,0.2530670,0.3235553,0.3943958,0.7973731,8.621081,37.23711,1.486982,-0.09404361,1.060612,-0.10349363
6,0.01124955,0.01239871,0.5616888,2.578908,0.2044972,0.1928170,0.4591589,1.658333,0.1515471,1.234711,⋯,0.2609731,0.3423476,0.4145336,0.9106479,8.084553,36.27321,1.438798,-0.08098956,1.066427,-0.08849283


This is actually enough to start training models.  We have the vegetation indices, but instead of doing that, let's transform the data and write it to file.  Then we will proceed to creating the model corrections, etc.

In [112]:
img_resampled_bands <- resample_df(img_bands, drop_existing=TRUE)# corrects scale difference (poorly)
grd_resampled_bands <- resample_df(0.01*grd_bands, drop_existing=TRUE)
head(img_resampled_bands)
head(grd_resampled_bands)

Using spline to predict value at new bands...

Beware the spectra are now partially smoothed.

Using spline to predict value at new bands...

Beware the spectra are now partially smoothed.



,X402.593_5nm,X407.593_5nm,X412.593_5nm,X417.593_5nm,X422.593_5nm,X427.593_5nm,X432.593_5nm,X437.593_5nm,X442.593_5nm,X447.593_5nm,⋯,X947.593_5nm,X952.593_5nm,X957.593_5nm,X962.593_5nm,X967.593_5nm,X972.593_5nm,X977.593_5nm,X982.593_5nm,X987.593_5nm,X992.593_5nm
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,0.06212404,0.06180653,0.05693654,0.05199935,0.04943319,0.04810434,0.04814491,0.04839997,0.04902924,0.05122705,⋯,0.7765625,0.6580751,0.5365841,0.5196822,0.6809964,0.8697628,0.9139739,0.8369247,0.7141234,0.6972216
2,0.05749476,0.06297455,0.05918532,0.05290789,0.05137537,0.05212780,0.05244476,0.05221252,0.05233717,0.05398912,⋯,0.7950162,0.6789633,0.6053080,0.6241000,0.7584388,0.9404683,1.0002115,0.9083915,0.7245195,0.6902076
3,0.04999572,0.04854917,0.04249125,0.03980269,0.04341043,0.04719443,0.04936688,0.05086627,0.05222365,0.05477898,⋯,0.7504691,0.7452264,0.6078203,0.5917521,0.8167859,0.9920923,1.0107302,0.8651387,0.5893411,0.5292330
4,0.05353346,0.04877967,0.05867343,0.04561053,0.04907692,0.04761848,0.04514567,0.04761746,0.05044233,0.04838392,⋯,0.8047186,0.7281673,0.4420411,0.4745701,0.8036229,0.8446532,0.8059575,0.8769885,0.6174775,0.5569951
5,0.06426926,0.06406918,0.05702433,0.05276894,0.04848679,0.04424580,0.04301885,0.04311201,0.04557514,0.04929684,⋯,0.7491398,0.7379520,0.6675188,0.6551306,0.7406565,0.8998303,0.9689108,0.8887908,0.7388946,0.6410263
6,0.06286275,0.05956411,0.05707863,0.05208934,0.04850874,0.04746565,0.04748539,0.04717902,0.04801729,0.05065653,⋯,0.7746607,0.6766340,0.5459924,0.5157587,0.6739425,0.8702676,0.9066815,0.8294759,0.7525363,0.7513382


,X402.593_5nm,X407.593_5nm,X412.593_5nm,X417.593_5nm,X422.593_5nm,X427.593_5nm,X432.593_5nm,X437.593_5nm,X442.593_5nm,X447.593_5nm,⋯,X947.593_5nm,X952.593_5nm,X957.593_5nm,X962.593_5nm,X967.593_5nm,X972.593_5nm,X977.593_5nm,X982.593_5nm,X987.593_5nm,X992.593_5nm
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,0.07245802,0.06661333,0.06098925,0.05743042,0.05707706,0.05856749,0.05834355,0.05622911,0.05524288,0.05607725,⋯,0.6369729,0.6345204,0.6313397,0.6287968,0.6281234,0.6295651,0.6328722,0.6368847,0.6405124,0.6433084
2,0.08115942,0.07483624,0.06916964,0.06574895,0.06593264,0.06811629,0.06807350,0.06562520,0.06441746,0.06535565,⋯,0.7760084,0.7687512,0.7606341,0.7543381,0.7519981,0.7533016,0.7572998,0.7624575,0.7674879,0.7720926
3,0.08136262,0.07301538,0.06552952,0.06120607,0.06086955,0.06284314,0.06328390,0.06174239,0.06098788,0.06176050,⋯,0.7843013,0.7789967,0.7726606,0.7676052,0.7658361,0.7674851,0.7719528,0.7775034,0.7825374,0.7863973
4,0.05896560,0.05189205,0.04545328,0.04166453,0.04150173,0.04348511,0.04398803,0.04261654,0.04204676,0.04303856,⋯,0.6211121,0.6184846,0.6150843,0.6122652,0.6112705,0.6123582,0.6152755,0.6188170,0.6219192,0.6243869
5,0.04558364,0.04143094,0.03767347,0.03563905,0.03578412,0.03713103,0.03746708,0.03664839,0.03648692,0.03749059,⋯,0.4052126,0.4035104,0.4017624,0.4005248,0.4001966,0.4007239,0.4019035,0.4033335,0.4047428,0.4063263
6,0.06398296,0.05639362,0.04960852,0.04582443,0.04563105,0.04742821,0.04774089,0.04627079,0.04572180,0.04679171,⋯,0.5233318,0.5183589,0.5136390,0.5102970,0.5088727,0.5083518,0.5078202,0.5074100,0.5075353,0.5088901


In [113]:
img_raw_with_na <- cbind(img_resampled_bands, img_indices)
grd_raw_with_na <- cbind(grd_resampled_bands, grd_indices)


In [114]:
img_raw <- impute_spectra(img_raw_with_na)
grd_raw <- impute_spectra(inf_to_na(grd_raw_with_na))# note also dropping an Inf (liekly div by 0 in veg index)

[1] "Imputing..."
[1] "Imputing..."


Apply the outlier transforms

In [115]:
grd_clipped <- clip_outliers(grd_raw)
grd_imputed <- impute_outliers_and_na(grd_raw)
grd_dropped <- grd_raw[detect_outliers_columnwise(grd_raw),]
img_clipped <- clip_outliers(img_raw)
img_imputed <- impute_outliers_and_na(img_raw)
img_dropped <- img_raw[detect_outliers_columnwise(img_raw),]

[1] "Imputing..."
[1] "Imputing..."
[1] "Imputing..."
[1] "Imputing..."


Now the center/scale transforms

In [116]:
grd_raw_robust <- columnwise_robust_scale(grd_raw)
img_raw_robust <- columnwise_robust_scale(img_raw)
grd_raw_minmax <- columnwise_min_max_scale(grd_raw)
img_raw_minmax <- columnwise_min_max_scale(img_raw)
grd_raw_standard <- standardize_df(grd_raw)
img_raw_standard <- standardize_df(img_raw)

grd_clipped_robust <- columnwise_robust_scale(grd_clipped)
grd_imputed_robust <- columnwise_robust_scale(grd_imputed)
grd_dropped_robust <- columnwise_robust_scale(grd_dropped)
img_clipped_robust <- columnwise_robust_scale(img_clipped)
img_imputed_robust <- columnwise_robust_scale(img_imputed)
img_dropped_robust <- columnwise_robust_scale(img_dropped)

grd_clipped_minmax <- columnwise_min_max_scale(grd_clipped)
grd_imputed_minmax <- columnwise_min_max_scale(grd_imputed)
grd_dropped_minmax <- columnwise_min_max_scale(grd_dropped)
img_clipped_minmax <- columnwise_min_max_scale(img_clipped)
img_imputed_minmax <- columnwise_min_max_scale(img_imputed)
img_dropped_minmax <- columnwise_min_max_scale(img_dropped)

grd_clipped_standard <- standardize_df(grd_clipped)
grd_imputed_standard <- standardize_df(grd_imputed)
grd_dropped_standard <- standardize_df(grd_imputed)
img_clipped_standard <- standardize_df(img_clipped)
img_imputed_standard <- standardize_df(img_imputed)
img_dropped_standard <- standardize_df(img_dropped)
#

Now, let's save all these data to disk

In [117]:
BASE_PATH <- "Data/gs/"
X_TRAIN_PATH <- paste0(BASE_PATH, "x_train/")
Y_TRAIN_PATH <- paste0(BASE_PATH, "y_train/")

X_TEST_PATH <- paste0(BASE_PATH, "x_test/")
Y_TEST_PATH <- paste0(BASE_PATH, "y_test/")

if(!dir.exists(BASE_PATH)){
    dir.create(BASE_PATH)
}
if(!dir.exists(X_TRAIN_PATH)){
    dir.create(X_TRAIN_PATH)
}
if(!dir.exists(Y_TRAIN_PATH)){
    dir.create(Y_TRAIN_PATH)
}
if(!dir.exists(X_TEST_PATH)){
    dir.create(X_TEST_PATH)
}
if(!dir.exists(Y_TEST_PATH)){
    dir.create(Y_TEST_PATH)
}


In [118]:
write.csv(grd_clipped, file=paste0(X_TRAIN_PATH, "grd_clipped_raw.csv"))
write.csv(grd_clipped_minmax, file=paste0(X_TRAIN_PATH, "grd_clipped_minmax.csv"))
write.csv(grd_clipped_robust, file=paste0(X_TRAIN_PATH, "grd_clipped_robust.csv"))
write.csv(grd_clipped_standard, file=paste0(X_TRAIN_PATH, "grd_clipped_standard.csv"))

write.csv(grd_imputed, file=paste0(X_TRAIN_PATH, "grd_imputed_raw.csv"))
write.csv(grd_imputed_minmax, file=paste0(X_TRAIN_PATH, "grd_imputed_minmax.csv"))
write.csv(grd_imputed_robust, file=paste0(X_TRAIN_PATH, "grd_imputed_robust.csv"))
write.csv(grd_imputed_standard, file=paste0(X_TRAIN_PATH, "grd_imputed_standard.csv"))

write.csv(grd_dropped, file=paste0(X_TRAIN_PATH, "grd_dropped_raw.csv"))
write.csv(grd_dropped_minmax, file=paste0(X_TRAIN_PATH, "grd_dropped_minmax.csv"))
write.csv(grd_dropped_robust, file=paste0(X_TRAIN_PATH, "grd_dropped_robust.csv"))
write.csv(grd_dropped_standard, file=paste0(X_TRAIN_PATH, "grd_dropped_standard.csv"))

write.csv(grd_raw, file=paste0(X_TRAIN_PATH, "grd_raw_raw.csv"))
write.csv(grd_raw_minmax, file=paste0(X_TRAIN_PATH, "grd_raw_minmax.csv"))
write.csv(grd_raw_robust, file=paste0(X_TRAIN_PATH, "grd_raw_robust.csv"))
write.csv(grd_raw_standard, file=paste0(X_TRAIN_PATH, "grd_raw_standard.csv"))

In [119]:
write.csv(img_clipped, file=paste0(X_TRAIN_PATH, "img_clipped_raw.csv"))
write.csv(img_clipped_minmax, file=paste0(X_TRAIN_PATH, "img_clipped_minmax.csv"))
write.csv(img_clipped_robust, file=paste0(X_TRAIN_PATH, "img_clipped_robust.csv"))
write.csv(img_clipped_standard, file=paste0(X_TRAIN_PATH, "img_clipped_standard.csv"))

write.csv(img_imputed, file=paste0(X_TRAIN_PATH, "img_imputed_raw.csv"))
write.csv(img_imputed_minmax, file=paste0(X_TRAIN_PATH, "img_imputed_minmax.csv"))
write.csv(img_imputed_robust, file=paste0(X_TRAIN_PATH, "img_imputed_robust.csv"))
write.csv(img_imputed_standard, file=paste0(X_TRAIN_PATH, "img_imputed_standard.csv"))

write.csv(img_dropped, file=paste0(X_TRAIN_PATH, "img_dropped_raw.csv"))
write.csv(img_dropped_minmax, file=paste0(X_TRAIN_PATH, "img_dropped_minmax.csv"))
write.csv(img_dropped_robust, file=paste0(X_TRAIN_PATH, "img_dropped_robust.csv"))
write.csv(img_dropped_standard, file=paste0(X_TRAIN_PATH, "img_dropped_standard.csv"))

write.csv(img_raw, file=paste0(X_TRAIN_PATH, "img_raw_raw.csv"))
write.csv(img_raw_minmax, file=paste0(X_TRAIN_PATH, "img_raw_minmax.csv"))
write.csv(img_raw_robust, file=paste0(X_TRAIN_PATH, "img_raw_robust.csv"))
write.csv(img_raw_standard, file=paste0(X_TRAIN_PATH, "img_raw_standard.csv"))

In [120]:
write.csv(grd_raw[,colnames(grd_indices)], file=paste0(X_TRAIN_PATH, "grd_indices_only.csv"))


## Labels for the above Data

In [121]:
img_targets <- img_speclib$FncGrp1 %>% as.factor()
grd_targets <- grd_speclib$Functional_group1 %>% as.factor()

In [122]:
write.csv(img_targets, file="Data/gs/y_train/img_indices_only.csv")
write.csv(grd_targets, file="Data/gs/y_train/grd_indices_only.csv")


In [123]:
img_targets %>% table()

.
       Abiotic      Graminoid         Lichen           Moss     ShrubDecid 
           797            145             97             92            107 
ShrubEvergreen  TreeBroadleaf    TreeConifer 
           137            100           2401 

In [124]:
grd_targets %>% table()

.
       Abiotic           Forb      Graminoid         Lichen           Moss 
            44             69              2            281             63 
    ShrubDecid ShrubEvergreen  TreeBroadleaf    TreeConifer 
           118             41             13             17 

In [125]:
# drop entries with outliers to match training data
img_targets_dropped <- img_targets[detect_outliers_columnwise(img_raw)]
grd_targets_dropped <- grd_targets[detect_outliers_columnwise(grd_raw)]

In [126]:
write.csv(grd_targets, file=paste0(Y_TRAIN_PATH, "grd_clipped_raw.csv"))
write.csv(grd_targets, file=paste0(Y_TRAIN_PATH, "grd_clipped_minmax.csv"))
write.csv(grd_targets, file=paste0(Y_TRAIN_PATH, "grd_clipped_robust.csv"))
write.csv(grd_targets, file=paste0(Y_TRAIN_PATH, "grd_clipped_standard.csv"))

write.csv(grd_targets, file=paste0(Y_TRAIN_PATH, "grd_imputed_raw.csv"))
write.csv(grd_targets, file=paste0(Y_TRAIN_PATH, "grd_imputed_minmax.csv"))
write.csv(grd_targets, file=paste0(Y_TRAIN_PATH, "grd_imputed_robust.csv"))
write.csv(grd_targets, file=paste0(Y_TRAIN_PATH, "grd_imputed_standard.csv"))

write.csv(grd_targets, file=paste0(Y_TRAIN_PATH, "grd_raw_raw.csv"))
write.csv(grd_targets, file=paste0(Y_TRAIN_PATH, "grd_raw_minmax.csv"))
write.csv(grd_targets, file=paste0(Y_TRAIN_PATH, "grd_raw_robust.csv"))
write.csv(grd_targets, file=paste0(Y_TRAIN_PATH, "grd_raw_standard.csv"))

#write.csv(grd_targets_dropped, file=paste0(Y_TRAIN_PATH, "grd_dropped_raw.csv"))
#write.csv(grd_targets_dropped, file=paste0(Y_TRAIN_PATH, "grd_dropped_minmax.csv"))
#write.csv(grd_targets_dropped, file=paste0(Y_TRAIN_PATH, "grd_dropped_robust.csv"))
#write.csv(grd_targets_dropped, file=paste0(Y_TRAIN_PATH, "grd_dropped_standard.csv"))

In [127]:
write.csv(img_targets, file=paste0(Y_TRAIN_PATH, "img_clipped_raw.csv"))
write.csv(img_targets, file=paste0(Y_TRAIN_PATH, "img_clipped_minmax.csv"))
write.csv(img_targets, file=paste0(Y_TRAIN_PATH, "img_clipped_robust.csv"))
write.csv(img_targets, file=paste0(Y_TRAIN_PATH, "img_clipped_standard.csv"))

write.csv(img_imputed, file=paste0(Y_TRAIN_PATH, "img_imputed_raw.csv"))
write.csv(img_targets, file=paste0(Y_TRAIN_PATH, "img_imputed_minmax.csv"))
write.csv(img_targets, file=paste0(Y_TRAIN_PATH, "img_imputed_robust.csv"))
write.csv(img_targets, file=paste0(Y_TRAIN_PATH, "img_imputed_standard.csv"))

write.csv(img_targets, file=paste0(Y_TRAIN_PATH, "img_raw_raw.csv"))
write.csv(img_targets, file=paste0(Y_TRAIN_PATH, "img_raw_minmax.csv"))
write.csv(img_targets, file=paste0(Y_TRAIN_PATH, "img_raw_robust.csv"))
write.csv(img_targets, file=paste0(Y_TRAIN_PATH, "img_raw_standard.csv"))
write.csv(img_targets_dropped, file=paste0(Y_TRAIN_PATH, "img_dropped_raw.csv"))
write.csv(img_targets_dropped, file=paste0(Y_TRAIN_PATH, "img_dropped_minmax.csv"))
write.csv(img_targets_dropped, file=paste0(Y_TRAIN_PATH, "img_dropped_robust.csv"))
write.csv(img_targets_dropped, file=paste0(Y_TRAIN_PATH, "img_dropped_standard.csv"))

## Test Data

Build the test data, and save it with the same names as the training data

In [128]:
set.seed(61718L)

permutation <-  permute::shuffle(length(img_targets))
sample <- create_stratified_sample(
    img_targets, 
    permutation = permutation,
    samples_per_pft = 15)
# split the data based on the above sample
img_targets_test <- img_targets[permutation][sample]
img_targets_train <- img_targets[permutation][-sample]
img_raw_test <- img_raw[permutation,][sample,]
img_raw_train <- img_raw[permutation,][-sample,]


[1] "Moss"        "Abiotic"     "ShrubDecid"  "TreeConifer" "Abiotic"    


In [129]:
img_targets_test %>% as.factor() %>% table()

.
       Abiotic      Graminoid         Lichen           Moss     ShrubDecid 
            15             15             15             15             15 
ShrubEvergreen  TreeBroadleaf    TreeConifer 
            15             15             15 

In [130]:
# create the subsampled data and save them for each processing type/treatment

# clipped
img_clipped_train <- img_clipped[permutation,][-sample,]
img_clipped_test <- img_clipped[permutation,][sample,]
img_clipped_minmax_train <- img_clipped_minmax[permutation,][-sample,]
img_clipped_minmax_test <- img_clipped_minmax[permutation,][sample,]
img_clipped_robust_train <- img_clipped_robust[permutation,][-sample,]
img_clipped_robust_test <- img_clipped_robust[permutation,][sample,]
img_clipped_standard_train <- img_clipped_standard[permutation,][-sample]
img_clipped_standard_test <- img_clipped_standard[permutation,][sample,]

# raw (note one is done in the previous cell)
img_raw_minmax_train <- img_raw_minmax[permutation,][-sample,]
img_raw_minmax_test <- img_raw_minmax[permutation,][sample,]
img_raw_robust_train <- img_raw_robust[permutation,][-sample,]
img_raw_robust_test <- img_raw_robust[permutation,][sample,]
img_raw_standard_train <- img_raw_standard[permutation,][sample,]
img_raw_standard_test <- img_raw_standard[permutation,][sample,]

#imputed
img_imputed_train <- img_imputed[permutation,][-sample,]
img_imputed_test <- img_imputed[permutation,][sample,]
img_imputed_minmax_train <- img_imputed_minmax[permutation,][-sample,]
img_imputed_minmax_test <- img_imputed_minmax[permutation,][sample,]
img_imputed_robust_train <- img_imputed_robust[permutation,][-sample,]
img_imputed_robust_test <- img_imputed_robust[permutation,][sample,]
img_imputed_standard_train <- img_imputed_standard[permutation,][-sample,]
img_imputed_standard_test <- img_imputed_standard[permutation,][sample,]



In [131]:
print(length(img_targets_test))
print(nrow(img_clipped_robust_test))

[1] 120
[1] 120


### Image-based Training Data

In [132]:
write.csv(img_clipped_train, file=paste0(X_TRAIN_PATH, "img_clipped_raw.csv"))
write.csv(img_clipped_minmax_train, file=paste0(X_TRAIN_PATH, "img_clipped_minmax.csv"))
write.csv(img_clipped_robust_train, file=paste0(X_TRAIN_PATH, "img_clipped_robust.csv"))
write.csv(img_clipped_standard_train, file=paste0(X_TRAIN_PATH, "img_clipped_standard.csv"))

write.csv(img_imputed_train, file=paste0(X_TRAIN_PATH, "img_imputed_raw.csv"))
write.csv(img_imputed_minmax_train, file=paste0(X_TRAIN_PATH, "img_imputed_minmax.csv"))
write.csv(img_imputed_robust_train, file=paste0(X_TRAIN_PATH, "img_imputed_robust.csv"))
write.csv(img_imputed_standard_train, file=paste0(X_TRAIN_PATH, "img_imputed_standard.csv"))

write.csv(img_dropped, file=paste0(X_TRAIN_PATH, "img_dropped_raw.csv"))
write.csv(img_dropped_minmax, file=paste0(X_TRAIN_PATH, "img_dropped_minmax.csv"))
write.csv(img_dropped_robust, file=paste0(X_TRAIN_PATH, "img_dropped_robust.csv"))
write.csv(img_dropped_standard, file=paste0(X_TRAIN_PATH, "img_dropped_standard.csv"))

write.csv(img_raw_train, file=paste0(X_TRAIN_PATH, "img_raw_raw.csv"))
write.csv(img_raw_minmax_train, file=paste0(X_TRAIN_PATH, "img_raw_minmax.csv"))
write.csv(img_raw_robust_train, file=paste0(X_TRAIN_PATH, "img_raw_robust.csv"))
write.csv(img_raw_standard_train, file=paste0(X_TRAIN_PATH, "img_raw_standard.csv"))

In [133]:
write.csv(img_targets_train, file=paste0(Y_TRAIN_PATH, "img_clipped_raw.csv"))
write.csv(img_targets_train, file=paste0(Y_TRAIN_PATH, "img_clipped_minmax.csv"))
write.csv(img_targets_train, file=paste0(Y_TRAIN_PATH, "img_clipped_robust.csv"))
write.csv(img_targets_train, file=paste0(Y_TRAIN_PATH, "img_clipped_standard.csv"))

write.csv(img_targets_train, file=paste0(Y_TRAIN_PATH, "img_imputed_raw.csv"))
write.csv(img_targets_train, file=paste0(Y_TRAIN_PATH, "img_imputed_minmax.csv"))
write.csv(img_targets_train, file=paste0(Y_TRAIN_PATH, "img_imputed_robust.csv"))
write.csv(img_targets_train, file=paste0(Y_TRAIN_PATH, "img_imputed_standard.csv"))

write.csv(img_dropped, file=paste0(X_TRAIN_PATH, "img_dropped_raw.csv"))
write.csv(img_dropped_minmax, file=paste0(X_TRAIN_PATH, "img_dropped_minmax.csv"))
write.csv(img_dropped_robust, file=paste0(X_TRAIN_PATH, "img_dropped_robust.csv"))
write.csv(img_dropped_standard, file=paste0(X_TRAIN_PATH, "img_dropped_standard.csv"))

write.csv(img_targets_train, file=paste0(Y_TRAIN_PATH, "img_raw_raw.csv"))
write.csv(img_targets_train, file=paste0(Y_TRAIN_PATH, "img_raw_minmax.csv"))
write.csv(img_targets_train, file=paste0(Y_TRAIN_PATH, "img_raw_robust.csv"))
write.csv(img_targets_train, file=paste0(Y_TRAIN_PATH, "img_raw_standard.csv"))

### Image Based Test Data
Note: this image-based test set is used for all the models (ground included)

In [134]:
write.csv(img_clipped_test, file=paste0(X_TEST_PATH, "img_clipped_raw.csv"))
write.csv(img_clipped_minmax_test, file=paste0(X_TEST_PATH, "img_clipped_minmax.csv"))
write.csv(img_clipped_robust_test, file=paste0(X_TEST_PATH, "img_clipped_robust.csv"))
write.csv(img_clipped_standard_test, file=paste0(X_TEST_PATH, "img_clipped_standard.csv"))

write.csv(img_imputed_test, file=paste0(X_TEST_PATH, "img_imputed_raw.csv"))
write.csv(img_imputed_minmax_test, file=paste0(X_TEST_PATH, "img_imputed_minmax.csv"))
write.csv(img_imputed_robust_test, file=paste0(X_TEST_PATH, "img_imputed_robust.csv"))
write.csv(img_imputed_standard_test, file=paste0(X_TEST_PATH, "img_imputed_standard.csv"))

write.csv(img_dropped, file=paste0(X_TEST_PATH, "img_dropped_raw.csv"))
write.csv(img_dropped_minmax, file=paste0(X_TEST_PATH, "img_dropped_minmax.csv"))
write.csv(img_dropped_robust, file=paste0(X_TEST_PATH, "img_dropped_robust.csv"))
write.csv(img_dropped_standard, file=paste0(X_TEST_PATH, "img_dropped_standard.csv"))

write.csv(img_raw_test, file=paste0(X_TEST_PATH, "img_raw_raw.csv"))
write.csv(img_raw_minmax_test, file=paste0(X_TEST_PATH, "img_raw_minmax.csv"))
write.csv(img_raw_robust_test, file=paste0(X_TEST_PATH, "img_raw_robust.csv"))
write.csv(img_raw_standard_test, file=paste0(X_TEST_PATH, "img_raw_standard.csv"))

### Ground test (from the images)

In [135]:
write.csv(img_clipped_test, file=paste0(X_TEST_PATH, "grd_clipped_raw.csv"))
write.csv(img_clipped_minmax_test, file=paste0(X_TEST_PATH, "grd_clipped_minmax.csv"))
write.csv(img_clipped_robust_test, file=paste0(X_TEST_PATH, "grd_clipped_robust.csv"))
write.csv(img_clipped_standard_test, file=paste0(X_TEST_PATH, "grd_clipped_standard.csv"))

write.csv(img_imputed_test, file=paste0(X_TEST_PATH, "grd_imputed_raw.csv"))
write.csv(img_imputed_minmax_test, file=paste0(X_TEST_PATH, "grd_imputed_minmax.csv"))
write.csv(img_imputed_robust_test, file=paste0(X_TEST_PATH, "grd_imputed_robust.csv"))
write.csv(img_imputed_standard_test, file=paste0(X_TEST_PATH, "grd_imputed_standard.csv"))

write.csv(img_dropped, file=paste0(X_TEST_PATH, "grd_dropped_raw.csv"))
write.csv(img_dropped_minmax, file=paste0(X_TEST_PATH, "grd_dropped_minmax.csv"))
write.csv(img_dropped_robust, file=paste0(X_TEST_PATH, "grd_dropped_robust.csv"))
write.csv(img_dropped_standard, file=paste0(X_TEST_PATH, "grd_dropped_standard.csv"))

write.csv(img_raw_test, file=paste0(X_TEST_PATH, "grd_raw_raw.csv"))
write.csv(img_raw_minmax_test, file=paste0(X_TEST_PATH, "grd_raw_minmax.csv"))
write.csv(img_raw_robust_test, file=paste0(X_TEST_PATH, "grd_raw_robust.csv"))
write.csv(img_raw_standard_test, file=paste0(X_TEST_PATH, "grd_raw_standard.csv"))

In [136]:
write.csv(img_targets_test, file=paste0(Y_TEST_PATH, "grd_clipped_raw.csv"))
write.csv(img_targets_test, file=paste0(Y_TEST_PATH, "grd_clipped_minmax.csv"))
write.csv(img_targets_test, file=paste0(Y_TEST_PATH, "grd_clipped_robust.csv"))
write.csv(img_targets_test, file=paste0(Y_TEST_PATH, "grd_clipped_standard.csv"))

write.csv(img_targets_test, file=paste0(Y_TEST_PATH, "grd_imputed_raw.csv"))
write.csv(img_targets_test, file=paste0(Y_TEST_PATH, "grd_imputed_minmax.csv"))
write.csv(img_targets_test, file=paste0(Y_TEST_PATH, "grd_imputed_robust.csv"))
write.csv(img_targets_test, file=paste0(Y_TEST_PATH, "grd_imputed_standard.csv"))

write.csv(img_targets_test, file=paste0(Y_TEST_PATH, "grd_raw_raw.csv"))
write.csv(img_targets_test, file=paste0(Y_TEST_PATH, "grd_raw_minmax.csv"))
write.csv(img_targets_test, file=paste0(Y_TEST_PATH, "grd_raw_robust.csv"))
write.csv(img_targets_test, file=paste0(Y_TEST_PATH, "grd_raw_standard.csv"))

write.csv(grd_targets_dropped, file=paste0(Y_TEST_PATH, "grd_dropped_raw.csv"))
write.csv(grd_targets_dropped, file=paste0(Y_TEST_PATH, "grd_dropped_minmax.csv"))
write.csv(grd_targets_dropped, file=paste0(Y_TEST_PATH, "grd_dropped_robust.csv"))
write.csv(grd_targets_dropped, file=paste0(Y_TEST_PATH, "grd_dropped_standard.csv"))

In [137]:
write.csv(img_targets_test, file=paste0(Y_TEST_PATH, "img_clipped_raw.csv"))
write.csv(img_targets_test, file=paste0(Y_TEST_PATH, "img_clipped_minmax.csv"))
write.csv(img_targets_test, file=paste0(Y_TEST_PATH, "img_clipped_robust.csv"))
write.csv(img_targets_test, file=paste0(Y_TEST_PATH, "img_clipped_standard.csv"))

write.csv(img_targets_test, file=paste0(Y_TEST_PATH, "img_imputed_raw.csv"))
write.csv(img_targets_test, file=paste0(Y_TEST_PATH, "img_imputed_minmax.csv"))
write.csv(img_targets_test, file=paste0(Y_TEST_PATH, "img_imputed_robust.csv"))
write.csv(img_targets_test, file=paste0(Y_TEST_PATH, "img_imputed_standard.csv"))

write.csv(img_targets_test, file=paste0(Y_TEST_PATH, "img_raw_raw.csv"))
write.csv(img_targets_test, file=paste0(Y_TEST_PATH, "img_raw_minmax.csv"))
write.csv(img_targets_test, file=paste0(Y_TEST_PATH, "img_raw_robust.csv"))
write.csv(img_targets_test, file=paste0(Y_TEST_PATH, "img_raw_standard.csv"))

write.csv(grd_targets_dropped, file=paste0(Y_TEST_PATH, "img_dropped_raw.csv"))
write.csv(grd_targets_dropped, file=paste0(Y_TEST_PATH, "img_dropped_minmax.csv"))
write.csv(grd_targets_dropped, file=paste0(Y_TEST_PATH, "img_dropped_robust.csv"))
write.csv(grd_targets_dropped, file=paste0(Y_TEST_PATH, "img_dropped_standard.csv"))

In [138]:
# need to write the targets fror training
clip_transform <- create_clip_transform(
    img_raw
)

Warning message in fences[column_name] <- list(upper = upper_fence, lower = lower_fence):
"number of items to replace is not a multiple of replacement length"
Warning message in fences[column_name] <- list(upper = upper_fence, lower = lower_fence):
"number of items to replace is not a multiple of replacement length"
Warning message in fences[column_name] <- list(upper = upper_fence, lower = lower_fence):
"number of items to replace is not a multiple of replacement length"
Warning message in fences[column_name] <- list(upper = upper_fence, lower = lower_fence):
"number of items to replace is not a multiple of replacement length"
Warning message in fences[column_name] <- list(upper = upper_fence, lower = lower_fence):
"number of items to replace is not a multiple of replacement length"
Warning message in fences[column_name] <- list(upper = upper_fence, lower = lower_fence):
"number of items to replace is not a multiple of replacement length"
Warning message in fences[column_name] <- list

In [139]:
save(clip_transform, file="./mle/clip_transform.rda")

In [140]:
clipped_2 <- clip_transform(img_raw)# clipped 2

## Sensor Correction

In this section, we create the models (and do some data transforms) to make the sensor-correction models and create the corrected data (only three times).  

We do this first for the raw (including outliers) data.

In [141]:
grd_resampled_to_match_img_bands <- resample_df(
    grd_bands,
    min_wavelength = 398,
    max_wavelength = 999,
    delta=1,
    drop_existing = TRUE
)
head(grd_resampled_to_match_img_bands)
head(img_bands)

,X398_5nm,X399_5nm,X400_5nm,X401_5nm,X402_5nm,X403_5nm,X404_5nm,X405_5nm,X406_5nm,X407_5nm,⋯,X990_5nm,X991_5nm,X992_5nm,X993_5nm,X994_5nm,X995_5nm,X996_5nm,X997_5nm,X998_5nm,X999_5nm
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,7.4884,7.5112,7.5454,7.5730,7.4709,7.2779,7.0436,6.8078,6.5930,6.4304,⋯,64.1969,64.2497,64.2992,64.3454,64.3884,64.4280,64.4644,64.4974,64.5272,64.5537
2,8.5251,8.4923,8.4576,8.4468,8.3543,8.1859,7.9498,7.6592,7.3842,7.2427,⋯,76.9688,77.0585,77.1472,77.2350,77.3218,77.4076,77.4925,77.5764,77.6594,77.7413
3,8.6621,8.6135,8.5510,8.4788,8.2999,8.0324,7.6948,7.3940,7.1467,6.9764,⋯,78.4546,78.5267,78.5944,78.6574,78.7159,78.7699,78.8193,78.8642,78.9045,78.9402
4,6.2702,6.2759,6.2892,6.2917,6.1648,5.9315,5.6163,5.3482,5.1269,4.9459,⋯,62.3163,62.3651,62.4116,62.4560,62.4980,62.5378,62.5754,62.6108,62.6439,62.6747
5,4.7252,4.7087,4.7035,4.7034,4.6276,4.4960,4.3321,4.1898,4.0757,4.0011,⋯,40.5451,40.5789,40.6136,40.6494,40.6861,40.7239,40.7628,40.8026,40.8434,40.8853
6,6.8568,6.8083,6.7481,6.7113,6.5617,6.3178,5.9993,5.7208,5.4961,5.3424,⋯,50.7966,50.8287,50.8664,50.9099,50.9590,51.0138,51.0742,51.1404,51.2122,51.2897


,Site,X398,X399,X400,X401,X402,X403,X404,X405,X406,⋯,X991,X992,X993,X994,X995,X996,X997,X998,X999,Area
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
1,BisonGulch,0.05814769,0.05926529,0.06028869,0.06116253,0.06183523,0.06229323,0.06254515,0.06259993,0.06246344,⋯,0.6811660,0.6890470,0.7040298,0.7249807,0.7507566,0.7801884,0.8121027,0.8453261,0.8786852,BisonGulch
2,BisonGulch,0.04456014,0.04778814,0.05079318,0.05356720,0.05609795,0.05833020,0.06018362,0.06157761,0.06245665,⋯,0.6683159,0.6786394,0.7000307,0.7308801,0.7695067,0.8140391,0.8625739,0.9132079,0.9640378,BisonGulch
3,BisonGulch,0.03929324,0.04265593,0.04557066,0.04787494,0.04942413,0.05025526,0.05051135,0.05033675,0.04985763,⋯,0.5091915,0.5178217,0.5395294,0.5726982,0.6156166,0.6663192,0.7227978,0.7830447,0.8450520,BisonGulch
4,BisonGulch,0.13230228,0.11122692,0.09129034,0.07379609,0.05996264,0.05014222,0.04418159,0.04192114,0.04302032,⋯,0.5113880,0.5348292,0.5745538,0.6227243,0.6723311,0.7185860,0.7570701,0.7833644,0.7930498,BisonGulch
5,BisonGulch,0.05211388,0.05565497,0.05878525,0.06139855,0.06339694,0.06476632,0.06554153,0.06575801,0.06545919,⋯,0.6680365,0.6509006,0.6344450,0.6181806,0.6017555,0.5851848,0.5685449,0.5519121,0.5353626,BisonGulch
6,BisonGulch,0.06955397,0.06788242,0.06631141,0.06486368,0.06356170,0.06242523,0.06147244,0.06072147,0.06017513,⋯,0.7371508,0.7445194,0.7567953,0.7732173,0.7930235,0.8154512,0.8397375,0.8651196,0.8908347,BisonGulch


In [142]:
img_b_names<- colnames(img_bands)
#img_b_names<-img_b_names[-c("Site","Area")]
img_b_names<-img_b_names[c(-1,-length(img_b_names))]
img_b_names
colnames(grd_resampled_to_match_img_bands) <-img_b_names
 

[1] "X398" "X399" "X400" "X401" "X402" "X403" "X404" "X405" "X406" "X407"
 [11] "X408" "X409" "X410" "X411" "X412" "X413" "X414" "X415" "X416" "X417"
 [21] "X418" "X419" "X420" "X421" "X422" "X423" "X424" "X425" "X426" "X427"
 [31] "X428" "X429" "X430" "X431" "X432" "X433" "X434" "X435" "X436" "X437"
 [41] "X438" "X439" "X440" "X441" "X442" "X443" "X444" "X445" "X446" "X447"
 [51] "X448" "X449" "X450" "X451" "X452" "X453" "X454" "X455" "X456" "X457"
 [61] "X458" "X459" "X460" "X461" "X462" "X463" "X464" "X465" "X466" "X467"
 [71] "X468" "X469" "X470" "X471" "X472" "X473" "X474" "X475" "X476" "X477"
 [81] "X478" "X479" "X480" "X481" "X482" "X483" "X484" "X485" "X486" "X487"
 [91] "X488" "X489" "X490" "X491" "X492" "X493" "X494" "X495" "X496" "X497"
[101] "X498" "X499" "X500" "X501" "X502" "X503" "X504" "X505" "X506" "X507"
[111] "X508" "X509" "X510" "X511" "X512" "X513" "X514" "X515" "X516" "X517"
[121] "X518" "X519" "X520" "X521" "X522" "X523" "X524" "X525" "X526" "X527"
[131] "X528" "X529" "X530" "X531" "X532" "X533" "X534" "X535" "X536" "X537"
[141] "X538" "X539" "X540" "X541" "X542" "X543" "X544" "X545" "X546" "X547"
[151] "X548" "X549" "X550" "X551" "X552" "X553" "X554" "X555" "X556" "X557"
[161] "X558" "X559" "X560" "X561" "X562" "X563" "X564" "X565" "X566" "X567"
[171] "X568" "X569" "X570" "X571" "X572" "X573" "X574" "X575" "X576" "X577"
[181] "X578" "X579" "X580" "X581" "X582" "X583" "X584" "X585" "X586" "X587"
[191] "X588" "X589" "X590" "X591" "X592" "X593" "X594" "X595" "X596" "X597"
[201] "X598" "X599" "X600" "X601" "X602" "X603" "X604" "X605" "X606" "X607"
[211] "X608" "X609" "X610" "X611" "X612" "X613" "X614" "X615" "X616" "X617"
[221] "X618" "X619" "X620" "X621" "X622" "X623" "X624" "X625" "X626" "X627"
[231] "X628" "X629" "X630" "X631" "X632" "X633" "X634" "X635" "X636" "X637"
[241] "X638" "X639" "X640" "X641" "X642" "X643" "X644" "X645" "X646" "X647"
[251] "X648" "X649" "X650" "X651" "X652" "X653" "X654" "X655" "X656" "X657"
[261] "X658" "X659" "X660" "X661" "X662" "X663" "X664" "X665" "X666" "X667"
[271] "X668" "X669" "X670" "X671" "X672" "X673" "X674" "X675" "X676" "X677"
[281] "X678" "X679" "X680" "X681" "X682" "X683" "X684" "X685" "X686" "X687"
[291] "X688" "X689" "X690" "X691" "X692" "X693" "X694" "X695" "X696" "X697"
[301] "X698" "X699" "X700" "X701" "X702" "X703" "X704" "X705" "X706" "X707"
[311] "X708" "X709" "X710" "X711" "X712" "X713" "X714" "X715" "X716" "X717"
[321] "X718" "X719" "X720" "X721" "X722" "X723" "X724" "X725" "X726" "X727"
[331] "X728" "X729" "X730" "X731" "X732" "X733" "X734" "X735" "X736" "X737"
[341] "X738" "X739" "X740" "X741" "X742" "X743" "X744" "X745" "X746" "X747"
[351] "X748" "X749" "X750" "X751" "X752" "X753" "X754" "X755" "X756" "X757"
[361] "X758" "X759" "X760" "X761" "X762" "X763" "X764" "X765" "X766" "X767"
[371] "X768" "X769" "X770" "X771" "X772" "X773" "X774" "X775" "X776" "X777"
[381] "X778" "X779" "X780" "X781" "X782" "X783" "X784" "X785" "X786" "X787"
[391] "X788" "X789" "X790" "X791" "X792" "X793" "X794" "X795" "X796" "X797"
[401] "X798" "X799" "X800" "X801" "X802" "X803" "X804" "X805" "X806" "X807"
[411] "X808" "X809" "X810" "X811" "X812" "X813" "X814" "X815" "X816" "X817"
[421] "X818" "X819" "X820" "X821" "X822" "X823" "X824" "X825" "X826" "X827"
[431] "X828" "X829" "X830" "X831" "X832" "X833" "X834" "X835" "X836" "X837"
[441] "X838" "X839" "X840" "X841" "X842" "X843" "X844" "X845" "X846" "X847"
[451] "X848" "X849" "X850" "X851" "X852" "X853" "X854" "X855" "X856" "X857"
[461] "X858" "X859" "X860" "X861" "X862" "X863" "X864" "X865" "X866" "X867"
[471] "X868" "X869" "X870" "X871" "X872" "X873" "X874" "X875" "X876" "X877"
[481] "X878" "X879" "X880" "X881" "X882" "X883" "X884" "X885" "X886" "X887"
[491] "X888" "X889" "X890" "X891" "X892" "X893" "X894" "X895" "X896" "X897"
[501] "X898" "X899" "X900" "X901" "X902" "X903" "X904" "X905" "X906" "X907"
[511] "X908" "X909" "X910" "X911" "X912" "X913" "X914" "X915" "X916" "X917"
[521] "X918" "X919" "X920" "X921" "X922" "X923" "X

In [143]:
grd_resampled_to_match_img_bands$targets <- grd_targets
img_bands_with_targets <- img_bands
img_bands_with_targets$targets <- img_targets

In [144]:
matched_data <- create_matched_data(
    img_bands_with_targets,
    grd_resampled_to_match_img_bands,
    cols=c("targets","targets")# assumes joining on columns named "targets" in each data.frame
)
head(matched_data$left)
head(matched_data$right)

,Site,X398,X399,X400,X401,X402,X403,X404,X405,X406,⋯,X992,X993,X994,X995,X996,X997,X998,X999,Area,targets
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<fct>
189,BisonGulch,0.1231361,0.1219915,0.1208316,0.1196679,0.1185103,0.1173540,0.1161858,0.1149919,0.1137625,⋯,0.2936604,0.2900522,0.2897067,0.2921721,0.2968943,0.3033019,0.3108238,0.3188887,BisonGulch,Abiotic
190,BisonGulch,0.1643986,0.1582972,0.1518950,0.1456436,0.1399717,0.1350771,0.1310226,0.1278694,0.1256291,⋯,0.1557894,0.1551025,0.1593620,0.1676876,0.1799044,0.1959548,0.2157815,0.2393268,BisonGulch,Abiotic
191,BisonGulch,0.1661341,0.1596000,0.1532859,0.1473391,0.1418970,0.1369964,0.1326156,0.1287321,0.1253150,⋯,0.1865305,0.2052519,0.2252224,0.2446971,0.2630286,0.2797515,0.2944007,0.3065108,BisonGulch,Abiotic
192,BisonGulch,0.1214014,0.1197652,0.1209832,0.1234412,0.1256133,0.1268757,0.1271308,0.1262880,0.1243613,⋯,0.1362033,0.1678828,0.2069852,0.2513454,0.3009331,0.3560723,0.4170872,0.4843018,BisonGulch,Abiotic
193,BisonGulch,0.1074620,0.1078207,0.1081793,0.1085379,0.1088966,0.1092552,0.1096139,0.1099726,0.1103313,⋯,0.2487216,0.2489848,0.2492481,0.2495115,0.2497750,0.2500386,0.2503022,0.2505658,BisonGulch,Abiotic
194,BisonGulch,0.1022341,0.1026307,0.1030274,0.1034240,0.1038207,0.1042175,0.1046143,0.1050112,0.1054082,⋯,0.2673160,0.2677757,0.2682356,0.2686955,0.2691555,0.2696155,0.2700755,0.2705355,BisonGulch,Abiotic


,X398,X399,X400,X401,X402,X403,X404,X405,X406,X407,⋯,X991,X992,X993,X994,X995,X996,X997,X998,X999,targets
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
18,12.5296,12.6062,12.6908,12.7803,12.7955,12.7510,12.6624,12.6126,12.5953,12.6003,⋯,18.4181,18.4210,18.4242,18.4277,18.4316,18.4358,18.4403,18.4452,18.4504,Abiotic
19,8.0423,8.0503,8.0688,8.1035,8.0919,8.0430,7.9695,7.8667,7.7654,7.7172,⋯,18.4996,18.5051,18.5103,18.5152,18.5197,18.5240,18.5279,18.5316,18.5349,Abiotic
20,7.8445,7.8459,7.8326,7.7877,7.7383,7.6777,7.5947,7.5238,7.4640,7.4080,⋯,21.0692,21.0950,21.1205,21.1458,21.1709,21.1956,21.2202,21.2445,21.2685,Abiotic
21,7.0172,7.0003,6.9872,7.0021,7.0185,7.0160,6.9667,6.9245,6.8957,6.8962,⋯,13.6072,13.6251,13.6433,13.6617,13.6805,13.6995,13.7188,13.7383,13.7581,Abiotic
22,6.4009,6.3791,6.3679,6.4081,6.3689,6.2896,6.2227,6.1535,6.1086,6.1448,⋯,17.2888,17.3038,17.3191,17.3347,17.3505,17.3666,17.3830,17.3996,17.4165,Abiotic
23,5.8715,5.9002,5.9289,5.9330,5.9219,5.8927,5.8412,5.7605,5.6801,5.6494,⋯,14.2387,14.2526,14.2674,14.2831,14.2998,14.3173,14.3358,14.3551,14.3754,Abiotic


In [145]:
correction_model <- build_columnwise_sensor_correction_model(
    matched_data$left,
    matched_data$right,
    grouping_variables =c("targets","targets")
)


Call:
lm(formula = left_vec ~ right_vec)

Residuals:
     Min       1Q   Median       3Q      Max 
-0.03371 -0.01691 -0.01508  0.01435  0.14095 

Coefficients:
             Estimate Std. Error t value Pr(>|t|)    
(Intercept) 0.0277306  0.0027816   9.969  < 2e-16 ***
right_vec   0.0022521  0.0002814   8.003 1.47e-14 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 0.029 on 382 degrees of freedom
Multiple R-squared:  0.1436,	Adjusted R-squared:  0.1413 
F-statistic: 64.05 on 1 and 382 DF,  p-value: 1.472e-14


Call:
lm(formula = left_vec ~ right_vec)

Residuals:
     Min       1Q   Median       3Q      Max 
-0.03231 -0.01675 -0.01503  0.01417  0.13337 

Coefficients:
             Estimate Std. Error t value Pr(>|t|)    
(Intercept) 0.0279014  0.0027258  10.236  < 2e-16 ***
right_vec   0.0022594  0.0002778   8.134 5.87e-15 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 0.02844 on 382 degree

In [146]:
print(correction_model)

$X398

Call:
lm(formula = left_vec ~ right_vec)

Coefficients:
(Intercept)    right_vec  
   0.027731     0.002252  


$X399

Call:
lm(formula = left_vec ~ right_vec)

Coefficients:
(Intercept)    right_vec  
   0.027901     0.002259  


$X400

Call:
lm(formula = left_vec ~ right_vec)

Coefficients:
(Intercept)    right_vec  
   0.027882     0.002279  


$X401

Call:
lm(formula = left_vec ~ right_vec)

Coefficients:
(Intercept)    right_vec  
   0.027727     0.002308  


$X402

Call:
lm(formula = left_vec ~ right_vec)

Coefficients:
(Intercept)    right_vec  
    0.02770      0.00235  


$X403

Call:
lm(formula = left_vec ~ right_vec)

Coefficients:
(Intercept)    right_vec  
   0.027836     0.002396  


$X404

Call:
lm(formula = left_vec ~ right_vec)

Coefficients:
(Intercept)    right_vec  
   0.028079     0.002441  


$X405

Call:
lm(formula = left_vec ~ right_vec)

Coefficients:
(Intercept)    right_vec  
   0.028270     0.002486  


$X406

Call:
lm(formula = left_vec ~ right_vec)


In [147]:
grd_corrected_bands <- apply_sensor_correction_model(
    correction_model,
    grd_resampled_to_match_img_bands,
    ignore_cols=c("targets")
)
head(grd_corrected_bands)

[1] "Correcting X398"
[1] "Correcting X399"
[1] "Correcting X400"
[1] "Correcting X401"
[1] "Correcting X402"
[1] "Correcting X403"
[1] "Correcting X404"
[1] "Correcting X405"
[1] "Correcting X406"
[1] "Correcting X407"
[1] "Correcting X408"
[1] "Correcting X409"
[1] "Correcting X410"
[1] "Correcting X411"
[1] "Correcting X412"
[1] "Correcting X413"
[1] "Correcting X414"
[1] "Correcting X415"
[1] "Correcting X416"
[1] "Correcting X417"
[1] "Correcting X418"
[1] "Correcting X419"
[1] "Correcting X420"
[1] "Correcting X421"
[1] "Correcting X422"
[1] "Correcting X423"
[1] "Correcting X424"
[1] "Correcting X425"
[1] "Correcting X426"
[1] "Correcting X427"
[1] "Correcting X428"
[1] "Correcting X429"
[1] "Correcting X430"
[1] "Correcting X431"
[1] "Correcting X432"
[1] "Correcting X433"
[1] "Correcting X434"
[1] "Correcting X435"
[1] "Correcting X436"
[1] "Correcting X437"
[1] "Correcting X438"
[1] "Correcting X439"
[1] "Correcting X440"
[1] "Correcting X441"
[1] "Correcting X442"
[1] "Corre

,X398,X399,X400,X401,X402,X403,X404,X405,X406,X407,⋯,X991,X992,X993,X994,X995,X996,X997,X998,X999,targets
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
1,0.04459509,0.04487255,0.04507446,0.04520287,0.04525520,0.04527101,0.04527079,0.04519668,0.04503515,0.04479759,⋯,0.5144720,0.5156872,0.5184141,0.5227222,0.5283796,0.5353075,0.5431197,0.5507648,0.5581383,ShrubDecid
2,0.04692982,0.04708930,0.04715299,0.04721934,0.04733075,0.04744618,0.04748258,0.04731357,0.04703311,0.04687129,⋯,0.5835198,0.5857119,0.5897141,0.5958124,0.6038053,0.6137759,0.6253021,0.6368393,0.6484344,ShrubDecid
3,0.04723836,0.04736314,0.04736581,0.04729319,0.04720294,0.04707846,0.04686019,0.04665419,0.04643336,0.04619146,⋯,0.5914344,0.5935995,0.5975822,0.6036909,0.6117217,0.6217672,0.6333940,0.6450003,0.6566434,ShrubDecid
4,0.04185160,0.04208145,0.04221211,0.04224600,0.04218652,0.04204563,0.04178715,0.04156759,0.04133291,0.04100783,⋯,0.5043127,0.5053994,0.5079627,0.5120391,0.5173955,0.5239300,0.5312652,0.5384208,0.5452727,ShrubDecid
5,0.03837214,0.03854045,0.03859896,0.03858066,0.03857487,0.03860681,0.03865278,0.03868739,0.03867839,0.03859587,⋯,0.3868708,0.3865950,0.3873376,0.3887741,0.3906328,0.3925525,0.3942326,0.3955302,0.3960797,ShrubDecid
6,0.04317267,0.04328439,0.04325775,0.04321431,0.04311904,0.04297103,0.04272195,0.04249401,0.04226523,0.04202005,⋯,0.4421240,0.4424752,0.4440944,0.4468290,0.4504284,0.4546582,0.4591906,0.4634921,0.4673191,ShrubDecid


In [148]:
grd_corrected_indices <- get_vegetation_indices(grd_corrected_bands, NULL)
head(grd_corrected_indices)

,Boochs,Boochs2,CARI,Carter,Carter2,Carter3,Carter4,Carter5,Carter6,CI,⋯,TCARI,TCARIOSAVI,TCARI2,TCARI2OSAVI2,TGI,TVI,Vogelmann,Vogelmann2,Vogelmann3,Vogelmann4
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,0.004534629,0.005193347,0.4177473,3.048996,0.3505744,0.2463269,0.5320798,1.583431,0.09575511,1.030801,⋯,0.1475220,0.2552669,0.2268021,0.7439430,2.835424,17.11559,1.332216,-0.07446616,1.1132502,-0.07913189
2,0.004624504,0.006129477,0.4215262,3.038753,0.3308106,0.2346899,0.4965629,1.515634,0.10907702,1.022980,⋯,0.1518209,0.2529217,0.2687585,0.7959817,3.668451,19.16085,1.379599,-0.08536271,1.2375053,-0.09146330
3,0.004821121,0.006892711,0.3808091,2.775697,0.2814817,0.2025133,0.4442394,1.499097,0.10165950,1.017261,⋯,0.1410058,0.2220896,0.2762434,0.7171465,3.379398,20.71182,1.469236,-0.10789825,1.2550949,-0.11723369
4,0.004755362,0.005098151,0.4412718,3.306291,0.3465766,0.2512668,0.5393252,1.684178,0.10802251,1.058908,⋯,0.1752781,0.2959170,0.2385932,0.7937309,4.391858,18.08078,1.317751,-0.07165548,1.0839585,-0.07601458
5,0.003262902,0.002851117,0.3702967,3.407217,0.4516263,0.3106945,0.6201746,1.511969,0.09340364,1.024622,⋯,0.1257463,0.2663865,0.1526728,0.7001264,2.789594,11.52202,1.234729,-0.05908405,0.9210774,-0.06152344
6,0.003781597,0.004007125,0.3971013,3.244449,0.3943057,0.2784809,0.5654268,1.576842,0.09572765,1.044258,⋯,0.1411034,0.2639490,0.1952926,0.7326426,2.984306,14.49382,1.295331,-0.07070474,1.0574192,-0.07453689


In [149]:
grd_corrected_resampled_bands <- resample_df(grd_corrected_bands, drop_existing=TRUE)
head(grd_corrected_resampled_bands)

Using spline to predict value at new bands...

Beware the spectra are now partially smoothed.



,X402.593_5nm,X407.593_5nm,X412.593_5nm,X417.593_5nm,X422.593_5nm,X427.593_5nm,X432.593_5nm,X437.593_5nm,X442.593_5nm,X447.593_5nm,⋯,X947.593_5nm,X952.593_5nm,X957.593_5nm,X962.593_5nm,X967.593_5nm,X972.593_5nm,X977.593_5nm,X982.593_5nm,X987.593_5nm,X992.593_5nm
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,0.04531129,0.04478104,0.04432200,0.04355032,0.04326464,0.04336750,0.04368233,0.04397782,0.04480150,0.04602156,⋯,0.5033286,0.4908440,0.4779830,0.4806951,0.5075264,0.5379154,0.5467201,0.5372123,0.5219957,0.5169601
2,0.04742543,0.04686660,0.04642778,0.04589219,0.04563124,0.04590262,0.04614415,0.04613729,0.04683548,0.04789975,⋯,0.5707320,0.5567622,0.5407387,0.5419051,0.5705575,0.6054651,0.6161106,0.6028518,0.5892978,0.5875488
3,0.04714641,0.04622051,0.04584184,0.04459883,0.04412326,0.04448649,0.04500232,0.04529865,0.04597860,0.04719940,⋯,0.5747677,0.5616491,0.5468959,0.5482080,0.5774462,0.6134598,0.6241719,0.6106847,0.5973236,0.5954092
4,0.04212631,0.04094861,0.04028378,0.03936130,0.03892313,0.03932589,0.04012739,0.04082608,0.04188188,0.04335356,⋯,0.4956013,0.4829522,0.4701278,0.4726841,0.4988781,0.5284919,0.5370560,0.5276836,0.5121153,0.5065983
5,0.03859924,0.03856140,0.03813532,0.03769906,0.03744018,0.03760402,0.03850910,0.03938558,0.04069580,0.04224634,⋯,0.3913400,0.3773448,0.3661187,0.3697388,0.3916879,0.4127230,0.4179434,0.4153124,0.3968636,0.3870085
6,0.04303465,0.04205028,0.04156559,0.04047835,0.03998726,0.04031861,0.04116230,0.04164669,0.04270840,0.04405596,⋯,0.4486267,0.4336254,0.4205378,0.4234487,0.4465214,0.4717958,0.4770798,0.4695383,0.4513917,0.4432654


In [150]:
write.csv(
    cbind(grd_corrected_resampled_bands, grd_corrected_indices), 
    file=paste0(X_TRAIN_PATH, "grd_raw_corrected.csv")
    )

# save labels also
write.csv(
    grd_resampled_to_match_img_bands$targets,
    file=paste0(Y_TRAIN_PATH, "grd_raw_corrected.csv"))

Now that that is done, we will move on to the clipped data

In [151]:
length(img_targets)
length(img_b_names)
img_bands

[1] 3876

[1] 602

,Site,X398,X399,X400,X401,X402,X403,X404,X405,X406,⋯,X991,X992,X993,X994,X995,X996,X997,X998,X999,Area
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
1,BisonGulch,0.05814769,0.05926529,0.06028869,0.06116253,0.06183523,0.06229323,0.06254515,0.06259993,0.06246344,⋯,0.6811660,0.6890470,0.7040298,0.7249807,0.7507566,0.7801884,0.8121027,0.8453261,0.8786852,BisonGulch
2,BisonGulch,0.04456014,0.04778814,0.05079318,0.05356720,0.05609795,0.05833020,0.06018362,0.06157761,0.06245665,⋯,0.6683159,0.6786394,0.7000307,0.7308801,0.7695067,0.8140391,0.8625739,0.9132079,0.9640378,BisonGulch
3,BisonGulch,0.03929324,0.04265593,0.04557066,0.04787494,0.04942413,0.05025526,0.05051135,0.05033675,0.04985763,⋯,0.5091915,0.5178217,0.5395294,0.5726982,0.6156166,0.6663192,0.7227978,0.7830447,0.8450520,BisonGulch
4,BisonGulch,0.13230228,0.11122692,0.09129034,0.07379609,0.05996264,0.05014222,0.04418159,0.04192114,0.04302032,⋯,0.5113880,0.5348292,0.5745538,0.6227243,0.6723311,0.7185860,0.7570701,0.7833644,0.7930498,BisonGulch
5,BisonGulch,0.05211388,0.05565497,0.05878525,0.06139855,0.06339694,0.06476632,0.06554153,0.06575801,0.06545919,⋯,0.6680365,0.6509006,0.6344450,0.6181806,0.6017555,0.5851848,0.5685449,0.5519121,0.5353626,BisonGulch
6,BisonGulch,0.06955397,0.06788242,0.06631141,0.06486368,0.06356170,0.06242523,0.06147244,0.06072147,0.06017513,⋯,0.7371508,0.7445194,0.7567953,0.7732173,0.7930235,0.8154512,0.8397375,0.8651196,0.8908347,BisonGulch
7,BisonGulch,0.06729458,0.06630754,0.06534554,0.06441492,0.06352106,0.06265933,0.06181928,0.06099038,0.06016029,⋯,0.7437618,0.7288627,0.7136456,0.6979235,0.6815704,0.6646246,0.6471513,0.6292161,0.6108842,BisonGulch
8,BisonGulch,0.06312026,0.06377031,0.06418433,0.06431848,0.06413609,0.06367344,0.06300940,0.06222334,0.06136778,⋯,0.6215974,0.6313994,0.6517848,0.6772707,0.7033992,0.7284635,0.7512134,0.7703991,0.7847703,BisonGulch
9,BisonGulch,0.06085542,0.05762401,0.05477036,0.05239887,0.05060432,0.04938326,0.04867495,0.04841791,0.04854619,⋯,0.6639090,0.6795856,0.6995446,0.7219862,0.7455690,0.7701834,0.7959240,0.8228855,0.8511626,BisonGulch


In [152]:
grd_resampled_to_img_clipped <- resample_df(
    clip_outliers(grd_bands),
    min_wavelength = 398,
    max_wavelength = 999,
    delta=1,
    drop_existing = TRUE
)

colnames(grd_resampled_to_img_clipped) <- img_b_names

grd_resampled_to_img_clipped$targets <- grd_targets
img_bands_with_targets <- img_bands
img_bands_with_targets$targets <- img_targets

matched_data_clipped <- create_matched_data(
    img_bands_with_targets,
    grd_resampled_to_img_clipped,
    cols=c("targets","targets")# assumes joining on columns named "targets" in each data.frame
)

correction_model <- build_columnwise_sensor_correction_model(
    matched_data_clipped$left,
    matched_data_clipped$right
)
grd_corrected_clipped_bands <- apply_sensor_correction_model(
    correction_model,
    grd_resampled_to_match_img_bands,
    ignore_cols=c("targets")
)
grd_corrected_clipped_indices <- get_vegetation_indices(grd_corrected_bands, NULL)
grd_corrected_clipped_resampled_bands <- resample_df(grd_corrected_bands, drop_existing=TRUE)





Call:
lm(formula = left_vec ~ right_vec)

Residuals:
     Min       1Q   Median       3Q      Max 
-0.03814 -0.02176 -0.01380  0.01416  0.15178 

Coefficients:
              Estimate Std. Error t value Pr(>|t|)    
(Intercept)  0.0504843  0.0043991  11.476   <2e-16 ***
right_vec   -0.0005109  0.0005362  -0.953    0.341    
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 0.0313 on 382 degrees of freedom
Multiple R-squared:  0.002371,	Adjusted R-squared:  -0.000241 
F-statistic: 0.9077 on 1 and 382 DF,  p-value: 0.3413


Call:
lm(formula = left_vec ~ right_vec)

Residuals:
     Min       1Q   Median       3Q      Max 
-0.03725 -0.02161 -0.01369  0.01453  0.14428 

Coefficients:
              Estimate Std. Error t value Pr(>|t|)    
(Intercept)  0.0497735  0.0042900  11.602   <2e-16 ***
right_vec   -0.0004083  0.0005253  -0.777    0.437    
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 0.03078 on 

Using spline to predict value at new bands...

Beware the spectra are now partially smoothed.



In [155]:
correction_model_r2<-lapply(1:length(correction_model), function(x) {summary(correction_model[[x]])$r.squared})
range(correction_model_r2)

[1] 2.445977e-06 1.959627e-01

In [154]:
write.csv(
    cbind(
        grd_corrected_clipped_indices,
        grd_corrected_clipped_resampled_bands
    ),
    file=paste0(
        X_TRAIN_PATH,
        "grd_clipped_corrected.csv"
    )
)

# save labels also
write.csv(
    grd_resampled_to_match_img_bands$targets,
    file=paste0(Y_TRAIN_PATH, "grd_clipped_corrected.csv"))

And finally the dropped outlier one

notes for later: should probably try PCA here.  clip -> scale -> PCA -> subset (and scale again for models like SVM and kNN)

...to be continued